In [1]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
import torch.nn.functional as F


from torch.utils.data import DataLoader, SubsetRandomSampler
from torch.nn import CrossEntropyLoss

import cv2
import numpy as np
import matplotlib.pyplot as plt

download mnist dataset using torchvision.datasets

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.1307, ),(0.3081, ))])

train_dataset = torchvision.datasets.MNIST('./mnist_data/',
                             download=True,
                             train=True,
                             transform=transform)

test_dataset = torchvision.datasets.MNIST("./mnist_data/",
                             download=False,
                             train=False,
                             transform= transform)

Extracting ./mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw


Extracting ./mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw



Extracting ./mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw


Extracting ./mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


split train and validation set using SubsetRandomSamplers (validation set is 10% of train_dataset)

In [3]:
validation_ratio=0.1
random_seed= 17

num_train = len(train_dataset)
indices = list(range(num_train))
split = int(np.floor(validation_ratio * num_train))

np.random.seed(random_seed)
np.random.shuffle(indices)

train_idx, valid_idx = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=128,
                                          sampler=train_sampler, num_workers=2)
validloader = torch.utils.data.DataLoader(train_dataset, batch_size=128,
                                          sampler=valid_sampler, num_workers=2)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=128,
                                         shuffle=False, num_workers=2)

In [4]:
print("train set : ",len(train_sampler))
print("validation set : ",len(valid_sampler))
print("test set : ",len(test_dataset))

train set :  54000
validation set :  6000
test set :  10000


모델 클래스 선언

In [10]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, 32)
        self.fc6 = nn.Linear(32, 10)
        self.drop = nn.Dropout(0.2)

    def forward(self, x):
        x = x.view(-1, 784)
        h1 = self.drop(F.relu(self.fc1(x)))
        h2 = self.drop(F.relu(self.fc2(h1)))
        h3 = self.drop(F.relu(self.fc3(h2)))
        h4 = self.drop(F.relu(self.fc4(h3)))
        h5 = self.drop(F.relu(self.fc5(h4)))
        h6 = self.fc6(h5)
        return h6

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

net = Net().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

cuda:0


training process

In [11]:
nb_epoch = 15

for epoch in range(nb_epoch):  

    net.train()
    
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


        running_loss += loss.item()
        if i % 2000 == 1999:    
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

    net.eval()
    correct = 0 
    total = 0

    with torch.no_grad():
        for data in validloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
    acc = correct / total     

print('Finished Training')

Finished Training


save trained model

In [12]:
PATH = './2015310908_임성규.pt'
torch.save(net.state_dict(), PATH)

load trained model

In [13]:
net = Net()
net.load_state_dict(torch.load(PATH))
net.eval()

Net(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=32, bias=True)
  (fc6): Linear(in_features=32, out_features=10, bias=True)
  (drop): Dropout(p=0.2, inplace=False)
)

test model

In [14]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of test dataset : %d %%' % (
    100 * correct / total))

Accuracy of test dataset : 97 %


In [15]:
from google.colab import drive 
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


save whole model

In [17]:
PATH2 = '/content/gdrive/My Drive/hw5/2015310908_임성규.pt'
torch.save(net, PATH2)